In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from datetime import datetime
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from concurrent.futures import ThreadPoolExecutor
from pymystem3 import Mystem

In [2]:
dfcolumns = pd.read_csv(r'C:/Users/msvic/ВКР/2Processed/POSTS_CENTR_1.0.csv',
                        nrows = 1, delimiter=';', encoding = 'UTF-8')
df = pd.read_csv(r'C:/Users/msvic/ВКР/2Processed/POSTS_CENTR_1.0.csv',
                  header = None,
                  delimiter=';',
                  skiprows = 1,
                  encoding='UTF-8',
                  usecols = list(range(len(dfcolumns.columns))),
                  names = dfcolumns.columns)

In [3]:
length = len(df)
print(length)

18197


In [23]:
#df.info()

In [22]:
#df.drop(df.columns [[0, 1, 2]], axis= 1 , inplace= True ) 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18197 entries, 0 to 18196
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              18197 non-null  int64  
 1   ТИП                             0 non-null      float64
 2   ПОДТИП                          0 non-null      float64
 3   ССЫЛКА НА ПОСТ                  18197 non-null  object 
 4   НАЗВАНИЕ ВЛАДЕЛЬЦА              18197 non-null  object 
 5   ПОДПИСЧИКОВ/ДРУЗЕЙ              18197 non-null  int64  
 6   ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ       18197 non-null  object 
 7   ССЫЛКА НА АВТОРА ПОСТА          18197 non-null  object 
 8   Unnamed: 8                      18197 non-null  int64  
 9   ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА  18197 non-null  object 
 10  ДАТА ПУБЛИКАЦИИ                 18197 non-null  object 
 11  ВРЕМЯ                           18197 non-null  object 
 12  ТЕКСТ                           

In [7]:
df

,ID,ТИП,ПОДТИП,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ПОДПИСЧИКОВ/ДРУЗЕЙ,ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ,ССЫЛКА НА АВТОРА ПОСТА,Unnamed: 8,ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА,...,?,...,ДЛИНА,ИМПЕРАТИВ,ПРОШЕДШЕЕ,ПОЗИТИВНОСТЬ,НЕЙТРАЛЬНОСТЬ,НЕГАТИВНОСТЬ,СЕНТИМЕНТ,ТОНАЛЬНОСТЬ
0,1,NaN,NaN,https://vk.com/wall-143265175_49172,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,796,0,3,0.000000,0.985506,0.038476,2,0.985506
1,2,NaN,NaN,https://vk.com/wall-143265175_49170,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,399,2,1,0.000000,0.988323,0.000000,2,0.988323
2,3,NaN,NaN,https://vk.com/wall-143265175_49169,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,0,0,0,0.000000,0.000000,0.000000,3,0.000000
3,4,NaN,NaN,https://vk.com/wall-143265175_49168,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,0,0,0,0.000000,0.000000,0.000000,3,0.000000
4,5,NaN,NaN,https://vk.com/wall-143265175_49164,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,261,0,1,0.037337,0.008587,0.000000,1,0.037337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18192,18193,NaN,NaN,https://vk.com/wall-182926490_9907,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,52,0,0,0.000000,0.685959,0.287778,2,0.685959
18193,18194,NaN,NaN,https://vk.com/wall-182926490_9905,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,88,0,0,0.000000,0.724880,0.843905,3,0.843905
18194,18195,NaN,NaN,https://vk.com/wall-182926490_9903,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,61,0,0,0.000000,0.658428,0.003604,2,0.658428
18195,18196,NaN,NaN,https://vk.com/wall-182926490_9901,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,0,0,0,0.000000,0.000000,0.000000,3,0.000000


In [8]:
cols = [col for col in df.columns]
print(cols)

['ID', 'ТИП', 'ПОДТИП', 'ССЫЛКА НА ПОСТ', 'НАЗВАНИЕ ВЛАДЕЛЬЦА', 'ПОДПИСЧИКОВ/ДРУЗЕЙ', 'ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ', 'ССЫЛКА НА АВТОРА ПОСТА', 'Unnamed: 8', 'ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА', 'ДАТА ПУБЛИКАЦИИ', 'ВРЕМЯ', 'ТЕКСТ', 'ИЛЛЮСТРАЦИИ', 'ЛАЙКОВ', 'РЕПОСТОВ', 'КОММЕНТАРИЕВ', 'ПРОСМОТРОВ', 'ВНЕШНИЙ_ИСТОЧНИК', 'ИСТОЧНИК', 'НАЗВАНИЕ ИСТОЧНИКА', 'ТЕКСТ2', 'ТЕКСТ3', '!', '?', '...', 'ДЛИНА', 'ИМПЕРАТИВ', 'ПРОШЕДШЕЕ', 'ПОЗИТИВНОСТЬ', 'НЕЙТРАЛЬНОСТЬ', 'НЕГАТИВНОСТЬ', 'СЕНТИМЕНТ', 'ТОНАЛЬНОСТЬ']


In [12]:
import pandas as pd
import openpyxl

# читаем файл Excel
wb = openpyxl.load_workbook('C:/Users/msvic/ВКР/Sources/Base.xlsx')
ws = wb.active


# получаем значения столбцов H, C и D - можно через ord, например: ord('C')-64
links = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=8, max_col=8) for cell in row] # H - 8, строки с 2 по 56
types = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=3, max_col=3) for cell in row]
subtypes = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=4, max_col=4) for cell in row]

# присваиваем значения столбца 'type' в зависимости от значения столбца 'link'
for i, row in df.iterrows():
    if row['ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ'] in links:
        index = links.index(row['ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ'])
        df.at[i, 'ТИП'] = types[index]
        df.at[i, 'ПОДТИП'] = subtypes[index]



In [13]:
df['ТЕКСТ2'] = df['ТЕКСТ']

In [ ]:
import re

print("Cleaning text...")
with open("C:/myR/stopwords.txt", "r", encoding="UTF-8") as file:
    stop_words = set(map(str.strip, file))


df['ТЕКСТ2'] = df['ТЕКСТ']
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.lower().str.replace("<[^>]*>", " ", regex=True) #Удаляем знаки
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\r|\n", "", regex=True) #Удаляем переходы на новую строку
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\[[^[]*\|", "", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[^\w\s]", " ", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("ё", "е", regex=True) #Заменяем "ё" на "е"
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\s\s+", " ", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\d", "", regex=True) #Удаляем цифры
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("  ", " ", regex=True) #Удаляем двойные пробелы
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.strip() #Удаляем пробелы в обоих концах строк
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[a-z]", "", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[A-Z]", "", regex=True)


# Удаляем стоп-слова
df['ТЕКСТ2'] = df['ТЕКСТ2'].apply(lambda x: ' '.join(filter(lambda word: word not in stop_words, x.split())) if isinstance(x, str) else x)

print("Successful!")
print("---------------------------------")



In [ ]:
df['ТЕКСТ2']

In [ ]:
df['ТЕКСТ3'] = df['ТЕКСТ2']

In [ ]:


### Здесь мы создаем функцию lemmatize_text, которая принимает на вход текст и возвращает лемматизированный текст 

def lemmatize_text(text):                         
    m = Mystem()
    lemmas = m.lemmatize(text)                       #лемматизируем текст
    list_len = len(lemmas)                           #узнаем количество слов в list'е lemmas (удвоенное)
    if list_len > 0:                                 #проверяем есть ли слова в посте
        if list_len == 2:                            #проверяем больше ли одного слова в посте
            mystr = lemmas[0]                        #приравниваем строку для записи в csv файл с единственным словом в посте
        elif list_len > 2:                           # == если в посте больше двух слов
            list_len -= 2                            # кол-во элементов в списке на 1 больше индекса последнего элемента списка;
            mystr = ""                               # последний элемент данного списка == '\n'
            i = 0
            while i <= list_len:                       #цикл считывания лемм
                mystr += lemmas[i] + " "               #прибавляем слова к строке для записи с пробелом на конце
                i += 2                               #так как в списке все слова имеют четный индекс, то прибавляем 2
        return mystr.strip()
    else:
        return ""                               


    
def process_chunk(chunk):                             # каждый chunk состояит из 1/n куска df
    
    for i, row in chunk.iterrows():
        text = row['ТЕКСТ2']
        if isinstance(text, str):
            lemmatized_text = lemmatize_text(text)
            df.at[i, 'ТЕКСТ3'] = lemmatized_text
            
      

    
def process_data_in_parallel(data, num_threads):               # распараллеливание процесса
    chunk_size = len(data) // num_threads
    chunks = [data.iloc[i:i+chunk_size] for i in range(0, len(data), chunk_size)]
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(process_chunk, chunks)


print("Lemmatizing text...") 
process_data_in_parallel(df, 4)        # 4 - n - количество параллельных процессов
print(' ')
print("Successful!")
print("---------------------------------")

In [ ]:
df['ТЕКСТ3']

In [ ]:
import nltk         #  pip install nltk
import pymorphy2    #  pip install pymorphy2 pymorphy2-dicts
#nltk.download('punkt')

In [ ]:
morph = pymorphy2.MorphAnalyzer()        #создать экземпляр класса MorphAnalyzer.


#Создать функцию count_imperative_verbs, которая будет принимать текст и экземпляр класса MorphAnalyzer, 
#а затем использовать его для определения частей речи и проверки, содержится ли глагол в повелительном наклонении.

def count_imperative_verbs(text):
    if isinstance(text, str):
        words = text.split()
        count = 0
        for word in words:
            parsed_word = morph.parse(word)[0]
            if 'VERB' in parsed_word.tag and 'impr' in parsed_word.tag:
                count += 1
        return count
    else:
        return 0

def count_past_verbs(text):
    if isinstance(text, str):
        words = text.split()
        count = 0
        for word in words:
            parsed_word = morph.parse(word)[0]
            if 'VERB' in parsed_word.tag and 'past' in parsed_word.tag:
                count += 1
        return count
    else:
        return 0


def count_punctuation(text, punctuation):
    if isinstance(text, str):
        return text.count(punctuation)
    else:
        return 0
    
def count_len(text):
    if isinstance(text, str):
        return len(text)
    else:
        return 0
        

df['!'] = df['ТЕКСТ'].apply(lambda x: count_punctuation(x, '!'))
df['?'] = df['ТЕКСТ'].apply(lambda x: count_punctuation(x, '?'))
df['...'] = df['ТЕКСТ'].apply(lambda x: count_punctuation(x, '...'))
df['ДЛИНА'] = df['ТЕКСТ3'].apply(lambda x: count_len(x))
df['ИМПЕРАТИВ'] = df['ТЕКСТ2'].apply(lambda x: count_imperative_verbs(x))
df['ПРОШЕДШЕЕ'] = df['ТЕКСТ2'].apply(lambda x: count_past_verbs(x))

In [ ]:
df.to_csv('POSTS_lemm1.csv', sep=';')

In [15]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [16]:
dat_text = df['ТЕКСТ'].tolist()
for i in range(len(dat_text)):
    if not isinstance(dat_text[i], str):
        dat_text[i] = ' '
results = model.predict(dat_text, k=3)

print("Successful!")
print("---------------------------------")

Successful!
---------------------------------


In [17]:
results[0]

{'neutral': 0.6001983880996704,
 'negative': 0.18714269995689392,
 'skip': 0.08036746829748154}

In [18]:
df.loc[:, ['ПОЗИТИВНОСТЬ', 'НЕЙТРАЛЬНОСТЬ', 'НЕГАТИВНОСТЬ', 'СЕНТИМЕНТ', 'ТОНАЛЬНОСТЬ']] = 0
numofrows = df.shape[0] #Количество строк в файле
dat_text = list(df['ТЕКСТ2'])

In [19]:
dat_pos = df['ПОЗИТИВНОСТЬ']
dat_neu = df['НЕЙТРАЛЬНОСТЬ']
dat_neg = df['НЕГАТИВНОСТЬ']

def process_row(i):
    if type(dat_text[i]) is str:
        try:
            df.loc[i, 'ПОЗИТИВНОСТЬ'] = results[i]['positive']
        except KeyError:
            df.loc[i, 'ПОЗИТИВНОСТЬ'] = 0
        try:
            df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = results[i]['neutral']
        except KeyError:
            df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = 0
        try:
            df.loc[i, 'НЕГАТИВНОСТЬ'] = results[i]['negative']
        except KeyError:
            df.loc[i, 'НЕГАТИВНОСТЬ'] = 0
    else:
        df.loc[i, 'ПОЗИТИВНОСТЬ'] = 0
        df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = 0
        df.loc[i, 'НЕГАТИВНОСТЬ'] = 0


    if (dat_pos[i] > dat_neg[i]):
        df.loc[i, 'СЕНТИМЕНТ'] = 1
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_pos[i]
    else:
        df.loc[i, 'СЕНТИМЕНТ'] = 3
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_neg[i]
    
        
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(process_row, range(numofrows))



print("\nSuccessful!")
print("---------------------------------")


Successful!
---------------------------------


In [20]:
df

,ID,ТИП,ПОДТИП,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ПОДПИСЧИКОВ/ДРУЗЕЙ,ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ,ССЫЛКА НА АВТОРА ПОСТА,Unnamed: 8,ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА,...,?,...,ДЛИНА,ИМПЕРАТИВ,ПРОШЕДШЕЕ,ПОЗИТИВНОСТЬ,НЕЙТРАЛЬНОСТЬ,НЕГАТИВНОСТЬ,СЕНТИМЕНТ,ТОНАЛЬНОСТЬ
0,1,official,dist,https://vk.com/wall-143265175_49172,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,796,0,3,0.000000,0.600198,0.187143,3,0.187143
1,2,official,dist,https://vk.com/wall-143265175_49170,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,399,2,1,0.294225,0.414909,0.182436,1,0.294225
2,3,official,dist,https://vk.com/wall-143265175_49169,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,0,0,0,0.000000,0.000000,0.000000,3,0.000000
3,4,official,dist,https://vk.com/wall-143265175_49168,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,0,0,0,0.000000,0.000000,0.000000,3,0.000000
4,5,official,dist,https://vk.com/wall-143265175_49164,Центральный район Санкт-Петербурга,10318,https://vk.com/club143265175,https://vk.com/club143265175,1,https://vk.com/club143265175?w=wall-143265175_...,...,0,0,261,0,1,0.262852,0.287778,0.098089,1,0.262852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18192,18193,public,nhbr,https://vk.com/wall-182926490_9907,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,52,0,0,0.000000,0.523430,0.300756,3,0.300756
18193,18194,public,nhbr,https://vk.com/wall-182926490_9905,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,88,0,0,0.000000,0.959164,0.314061,3,0.314061
18194,18195,public,nhbr,https://vk.com/wall-182926490_9903,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,61,0,0,0.000000,0.946607,0.073706,3,0.073706
18195,18196,public,nhbr,https://vk.com/wall-182926490_9901,Наш Дом — Лиговка-Ямская,1563,https://vk.com/club182926490,https://vk.com/club182926490,1,https://vk.com/club182926490?w=wall-182926490_...,...,0,0,0,0,0,0.000000,0.000000,0.000000,3,0.000000


In [21]:
# сохранение измененного файла csv
df.to_csv(r'C:/Users/msvic/ВКР/2Processed/POSTS_CENTR_2.0.csv', sep=';')